# Necessary imports

In [ ]:
%pip install scikit-learn
%pip install pandas

import pandas

# Tools
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier

# Definindo constantes

In [ ]:
RANDOM_SEED = 1337
N_JOBS = -1
FOLDS = 10

# Carregamento dos dados

In [ ]:
def load_data(file_path: str, positive_label_multiplication: int = 2) -> pandas.DataFrame:
    """ Receives a file path for the dataset training, testing and validation datasets. """

    # Loading data from csv file
    df = pandas.read_csv(file_path)

    # Selecting useful features
    useful_features = [
        "Bidder_Tendency",
        "Bidding_Ratio",
        "Successive_Outbidding",
        "Last_Bidding",
        "Auction_Bids",
        "Starting_Price_Average",
        "Early_Bidding",
        "Winning_Ratio",
        "Auction_Duration",
        "Class"
    ]

    df = df[useful_features]

    # Augmenting positive label data
    positive_labels = df[df["Class"] == 1]

    dfs_to_concat = [df]
    for _ in range(positive_label_multiplication):
        dfs_to_concat.append(positive_labels)

    df = pandas.concat(dfs_to_concat)
    df = df.sample(frac=1)

    # Separating features and labels
    columns = list(df.columns)
    features = columns[:len(columns)-1]
    label = columns[len(columns)-1:]

    X = df[features]
    y = df[label]

    # Creating training, testing and validation datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = RANDOM_SEED)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.125, random_state = RANDOM_SEED)

    return X_train, y_train, X_test, y_test, X_valid, y_valid

In [ ]:
file_path = "./dataset/Shill Bidding Dataset.csv"

X_train, y_train, X_test, y_test, X_valid, y_valid = load_data(file_path)

# Declarão de função para treinamento de modelos

A ferramenta gridsearch será usada para determinar a melhor combinação de parâmetros para cada modelo.

Uma função search é definida, que recebe uma instância de modelo a ser "variado" e um dicionário de parâmetros e suas variações. 

In [ ]:
def search(model, model_parameters, verbosity: int = 1): 
    """ """
    model_pipeline = Pipeline([
        ('clf', model)
    ])

    grid_search = GridSearchCV(
        model_pipeline,
        model_parameters,
        n_jobs=N_JOBS,
        cv=FOLDS,
        verbose=verbosity
    )

    grid_search.fit(X_train, y_train.values.ravel())

    print(f"Best score after optimization: {grid_search.best_score_}")
    print("Best params:")
    for key, value in grid_search.best_params_.items():
        print(f"{key}: {value}")

    return grid_search

# Variando K-NN

Default para leaf_size é 30, então decidimos um range de 20 a 40
p=0-> minkowski_distance,  p=1-> Manhattan distance, p=2-> Euclidean distance

In [ ]:
knn_parameters = {
    "clf__n_neighbors":range (4,10),
    "clf__weights":["uniform", "distance"],
    "clf__algorithm":["auto", "ball_tree", "kd_tree", "brute"],
    "clf__leaf_size":range(20,40),
    "clf__metric":["cityblock","cosine","euclidean","haversine","nan_euclidean"],
    "clf__p":range(1,3)
}

gs_knn = search(KNeighborsClassifier(), knn_parameters)

# Variando LVQ

# Variando Decision Tree

Max_depth começa com 3 porque é o minímo recomendado pelo Scikitlearn e vai até 50 porque raramente é necessário mais que 50. 
Min_samples_leaf foi recomendado ser entre 1 e 20

In [ ]:
dt_parameters = {
    "clf__criterion": ["gini", "entropy", "log_loss"],
    "clf__splitter": ["best", "random"],
    "clf__max_features": [0.2,0.4,0.6,0.8, None, "sqrt", "log2"],
    "clf__max_depth": range(3, 50),
    "clf__min_samples_split": range(2, 10),
    "clf__min_samples_leaf": range (1,20)

}

gs_dt = search(DecisionTreeClassifier(), dt_parameters)

# Variando SVM

Como há muito mais exemplos da classe 0 que a classe 1, o peso dado aos exemplos da classe 1 foi maior para 1 melhor treinamento. 
O C foi entre 0.1 e 1.0 pois foram vistos exemplos de outros modelos sendo treinados e esses eram os valores vistos.
O gamma é o coeficiente do kernel e o C o parâmetro de regularização. 

In [ ]:
svm_parameters = {
    "clf__kernel": ["linear", "poly", "rbf", "sigmoid"],
    "clf__gamma":["scale", "auto"],
    "clf__C": [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    "clf__degree":range (1,4), #degree of polinomial function
    "clf__class_weight":[{0: 2, 1: 8}, "balanced"],
    "clf__decision_function_shape":["ovr","ovo"] #one-vs-rest,one-vs-one
}

gs_svm = search(SVC(), svm_parameters)

# Variando Random Forest
Parâmetros e ranges escolhidos para a variação:
1. n_estimators [50:150]: Número de árvores de decisão presentes na floresta.
2. criterion ["gini", "entropy", "log_loss"]: Algoritmo para medição de qualidade de divisão de nós. Servem para determinar quais as melhores features para estarem mais perto do topo do árvore (ou seja, que apresentam maior ganho de informação).
3. max_depth [3:50]: Profundida máxima da árvore. O próprio SKLearn recomenda 3 como mínimo. O máximo depende muito das situações, mas via de regra, uma árvore mais profunda representa um classificador com risco de overfitting. Manter uma profundidade balanceada pode garantir um modelo mais genérico que consegue lidar com registros novos (diferentes dos que foram usado para o treino) sem muitos problemas.
4. max_features ["sqrt", "log2", None]: Número de atributos a serem considerados na hora de dividir.

In [ ]:
rf_parameters = {
    "clf__n_estimators": range(75, 125),
    "clf__criterion": ["gini", "entropy", "log_loss"],
    "clf__max_depth": range(3, 20),
    "clf__max_features": [0.2, 0.4, 0.6, 0.8, None, "sqrt", "log2"]
}

gs_rf = search(RandomForestClassifier(random_state=RANDOM_SEED), rf_parameters)

# Variando Rede Neural MLP
Parâmetros e ranges escolhidos para a variação:
1. hidden_layer_sizes [(100,), (50, 50,), (33, 33, 34,), (25, 25, 25, 25,)]: Quantidade de camadas internas do perceptron (e o número de neurons em cada uma)
2. Activation ["identity", "logistic", "tanh", "relu"]: Função de ativação para os neurons. 
3. Solver ["lbfgs", "sgd"]: De acordo com o sklearn, o solver "adam" funciona melhor com datasets maiores (na casa das dezenas de milhares). Como nosso dataset tem uma escala menor, seu uso não é recomendado, pois as alternativas convergem mais rapidamente e performam melhor.
4. Alpha: [0.0001, 0.0002, 0.0003, ..., 0.0009]: Termo de Regularização L2.
5. Learning rate ["constant", "invscaling", "adaptative"]: Taxa de aprendizado que define as atualizações de peso.
6. max_iter [50..1000]: Número de iterações a ocorrem internamente durante o treinamento. Maiores números significam mais tempo de processamento, e possivelmente mais precisão, já que o modelo terá mais tentativas para convergir numa solução.

In [ ]:
mlp_parameters = {
    "clf__hidden_layer_sizes": [(100,), (50, 50,), (33, 33, 34,), (25, 25, 25, 25,)],
    "clf__activation": ["identity", "logistic", "tanh", "relu"],
    "clf__solver": ["lbfgs", "sgd", "adam"],
    # "clf__alpha": [x/10000 for x in range(1, 10)],
    "clf__learning_rate": ["constant", "invscaling", "adaptative"],
    # "clf__max_iter": range(50, 1000, 50)
}

gs_mlp = search(MLPClassifier(random_state=RANDOM_SEED), mlp_parameters, 0)

# Variando Comitê de redes neurais Artificiais
Como o algoritmo Bagging é um ensemble e pode receber vários classificadores, decidimos instanciar o MLP com o resultado de sua otimização, para garantir maior qualidade do comitê.
Parâmetros e ranges escolhidos para a variação:
1. n_estimators [5:15]: Número de classificadores presentes no comitê.
2. max_samples [0.2:1]: Número de registros a serem extraídos de X na hora de treinar cada classificador do comitê.
3. max_features [0.2:1]: Número de atributos a serem considerados na hora de treinar cada classificador do comitê.
4. bootstrap [True, False]: .
5. bootstrap_features [True, False]: .

In [ ]:
ensemble_mlp_parameters = {
    "clf__n_estimators": range(5, 15),
    "clf__max_samples": [0.2, 0.4, 0.6, 0.8, 1],
    "clf__max_features": [0.2, 0.4, 0.6, 0.8, 1],
    "clf__bootstrap": [True, False],
    "clf__bootstrap_features": [True, False],
}

ideal_mlp = MLPClassifier(
    hidden_layer_sizes=(50, 50),
    learning_rate="constant",
    activation="logistic",
    solver="lbfgs",
    random_state=RANDOM_SEED,
    verbose=0
)

gs_ensemble_mlp = search(BaggingClassifier(ideal_mlp, random_state=RANDOM_SEED), ensemble_mlp_parameters)

# Variando Comitê Heterogêneo
Como o algoritmo VotingClassifier é um ensemble e pode receber vários classificadores diferentes, decidimos instanciá-los já com os melhores parâmetros provenientes de suas otimizações. Os classificadores escolhidos para o VotingClassifier foram:
- MLP
- Decision Tree
- SVM
- KNN

Parâmetros e ranges escolhidos para a variação:
1. voting ["hard", "soft"]: Regra para definir vencedor da votação (maioria simples com hard, probabilidade de classes com soft).
2. weights [None, [2.5, 2, 1.5, 1]]: pesos atribuídos para cada classificador. No caso de None, o peso é uniforme. Caso contrário, os pesos passados são atribuídos. Declaramos os classificadores em ordem do mais preciso para o menos preciso, garantindo que os melhores classificadores influenciem mais na decisão.

In [ ]:
voting_ensemble_parameters = {
    "clf__voting": ["hard", "soft"],
    "clf__weights": [None, [2.5, 2, 1.5, 1]]
}

estimators = [
    ("mlp", MLPClassifier(
        hidden_layer_sizes=(33, 33, 34,),
        activation="tanh",
        solver="lbfgs",
        random_state=RANDOM_SEED
    )),
    ("dt", DecisionTreeClassifier(
        criterion="gini",
        max_depth=38,
        max_features=None,
        splitter="random",
        random_state=RANDOM_SEED
    )),
    ("svm", SVC(
        C= 1.0,
        degree= 1,
        gamma= "auto",
        kernel= "rbf",
        random_state=RANDOM_SEED
    )),
    ("knn", KNeighborsClassifier(
        algorithm="auto",
        leaf_size=20,
        n_neighbors=6,
        p=2,
        weights="distance"
    ))
]

voting_ensemble = search(VotingClassifier(estimators), voting_ensemble_parameters)